# Homework notebook I - Mauricio Vega

This is the first notebook to help you practice your macroeconomic modelling and problem solving skills. It will also be part of your evaluation. Although I prefer, and highly recommend, to solve these tasks in **python**, I will accept any other programming language, as only solutions to the problems will be evaluated.

The notebook consist of the solution of one model, whose balanced growth path needs to be computed numerically:

- Using local approximations

- Using dynamic programing

## The model
Consider the next planning problem of a *dynastic* version of the neoclassical growth model

$$\max_{c_{t}, k_{t+1}, l_{t}}\sum^{\infty}_{t=0}\beta^{t}N_{t}\left\{\log(c_{t})+\mu\frac{(1-l_{t})^{1-\sigma}}{1-\sigma}\right\}$$

subject to:

$$c_{t}+k_{t+1}-(1-\delta)k_{t}=A_{t}k^{\alpha}_{t}l^{1-\alpha}_{t}$$

where $A_{t+1}=(1+\hat{A})A_{t}$ and $N_{t+1}=(1+\hat{N})N_{t}$ are the exogenous growth processes for productivity and population, and $\sigma\geq 0$

### Local approximations

#### Efficiency and feasibility 
Write down, in the next markdown cell, the efficiency conditions and the feasibility constraint that are needed for the solution of the problem:

#### Balanced growth path
- Write down, in the next markdown cell, the equations that allow for the solution of the economy in its balanced growth path

- Supose that $\beta=.97$, $\mu=1.5$, $\sigma=2.5$, $\alpha=.3$, $\delta=.08$, $\hat{A}=.025$, $\hat{N}=.015$, compute labor in the balanced growth path

Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.optimize import fsolve
from scipy.optimize import approx_fprime


- Plot labor supply in the balance growth path as a function of $\hat{A}\in \left[.01,.04\right]$ and give an intuitive explanation of your results  

#### Log-linearization
- Write down, in the next markdown cell, the log-linearized system of efficiency and feasibility constraints that allow for the application of the undetermined coefficients model

- Use the undetermined coefficient method to plot the labor and consumption policy functions 

#### Calibration
Use annual data for the US post-war period to find $\beta$, $\mu$. Keep the rest of the parameters set to the already stablished values. 

#### Simulation 
Use the policy functions for the new calibrated parameters to study the convergence trayectory of consumption and labor supply to a permanent productivity slowdown to a productivity growth $\hat{A}'=.012$ and give an intuitive explanation of your results, keep the rest of the parameters constant

- You may use the perfect foresight algorithm in Dynare

### Dynamic programming 

#### Bellman equation
Write down the Bellman equation of the planning problem in the next markdown cell

#### Value function iteration
Implement the value function iteration algorithm to compute the policy functions for consumption and labor, and plot them. Use the last set of parameters you found. 

#### Simulation
Use the policy funtions to simulate the same productivity slowdown than in the local approximation 

### Decentralized value function iteration

#### The small-k big-k algorithm
Use the algorithm that we studied in class to compute a decentralized recursive version of this economy, and plot the consumption and labor supply policy functions over aggregate capital  